In [8]:
from typing import List, Tuple
from datetime import datetime, timedelta
import numpy.typing as npt
import numpy as np
import os
import cv2
import pickle
from tqdm import tqdm
import itertools
import json

from utils.compose_factory import compose_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos

In [9]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/Users/chanwutk/Documents/apperception/data/v1.0-mini
/Users/chanwutk/Documents/apperception/data/v1.0-mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [10]:
fps = 20

In [11]:
interpolate = False

In [12]:
partition_dir = os.path.join(processed_dir, 'partition')

In [13]:
compose = compose_factory(fps, raw_dir, processed_dir, interpolate)

In [15]:
for l in os.listdir(partition_dir):
    d = os.path.join(partition_dir, l)
    with open(os.path.join(d, 'sample_data.pickle'), "rb") as f:
        sample_data = pickle.load(f)
    videos = get_videos(sample_data)
    
    
    output = {}
    for name in videos:
        video = videos[name]
        frames, filename, start = compose(l, name, video)
        output[name] = {
            "filename": filename,
            "start": start,
            "columns": [
                "cameraId",
                "frameId",
                "frameNum",
                "filename",
                "cameraTranslation",
                "cameraRotation",
                "cameraIntrinsic",
                "egoTranslation",
                "egoRotation",
                "timestamp",
                "cameraHeading",
                "egoHeading",
            ],
            "frames": video
        }
        print("done", filename)
    base = os.path.join(processed_dir, 'videos', l)
    with open(os.path.join(base, 'frames.pickle'), "wb") as f:
        pickle.dump(output, f)
    
    with open(os.path.join(base, 'frames.json'), "w") as f:
        json.dump(output, f, indent=2, sort_keys=True, default=str)

Reading scene: scene-0103-CAM_FRONT_LEFT


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:03<00:00, 128.74it/s]


Writing scene (/Users/chanwutk/Documents/apperception/data/v1.0-mini/videos/boston-seaport/scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4):


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:03<00:00, 116.08it/s]


done scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4
Reading scene: scene-0103-CAM_FRONT


...

done scene-1100-CAM_BACK-2018-11-21-3-49-47-937525.mp4
Reading scene: scene-1100-CAM_BACK_LEFT


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:03<00:00, 109.38it/s]


Writing scene (/Users/chanwutk/Documents/apperception/data/v1.0-mini/videos/singapore-hollandvillage/scene-1100-CAM_BACK_LEFT-2018-11-21-3-49-47-947423.mp4):


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:03<00:00, 103.12it/s]


done scene-1100-CAM_BACK_LEFT-2018-11-21-3-49-47-947423.mp4
